#  在 MLFlow 上运行 Phi-3 模型

本笔记本以 Phi-3 mini 4K Instruct 为例，介绍了如何使用 Transformer pipeline。

In [1]:
# 导入所需的库
import mlflow
import transformers

## Selecting Phi-3 Mini 4K from HuggingFace

In [2]:
# 在 HuggingFace 上选择 Phi-3 模型
pipeline = transformers.pipeline(
    task = "text-generation",
    model = "microsoft/Phi-3-mini-4k-instruct"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 生成 MLFlow 工件

In [3]:
# 设置 MLflow 模型配置
model_config = {
    "max_length": 300,
    "truncation": True,
    "include_prompt": False,
}

In [ ]:
#  从 HuggingFace model 生成 MLflow 工件
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model = pipeline,
        artifact_path = "phi3-mlflow-model",
        model_config = model_config,
        task = "llm/v1/chat"
    )

## 作为 MLFlow 模型运行 Phi-3

In [5]:
# Loading Phi-3 MLFlow model
loaded_model = mlflow.pyfunc.load_model(
    model_uri = model_info.model_uri
)

Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
loaded_model.metadata.signature

inputs: 
  ['messages': Array({content: string (required), name: string (optional), role: string (required)}) (required), 'temperature': double (optional), 'max_tokens': long (optional), 'stop': Array(string) (optional), 'n': long (optional), 'stream': boolean (optional)]
outputs: 
  ['id': string (required), 'object': string (required), 'created': long (required), 'model': string (required), 'choices': Array({finish_reason: string (required), index: long (required), message: {content: string (required), name: string (optional), role: string (required)} (required)}) (required), 'usage': {completion_tokens: long (required), prompt_tokens: long (required), total_tokens: long (required)} (required)]
params: 
  None

In [7]:
# 测试 MLFlow Phi-3 模型的推理能力
messages = [{"role": "user", "content": "What is the capital of Spain?"}]
response = loaded_model.predict(
    {"messages": messages}
)
print(response)

You are not running the flash-attention implementation, expect numerical differences.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'id': 'f9c2ca5c-b684-4a77-8a43-356b671fd797', 'object': 'chat.completion', 'created': 1718709547, 'model': 'microsoft/Phi-3-mini-4k-instruct', 'usage': {'prompt_tokens': 11, 'completion_tokens': 73, 'total_tokens': 84}, 'choices': [{'index': 0, 'finish_reason': 'stop', 'message': {'role': 'assistant', 'content': 'The capital of Spain is Madrid. It is the largest city in Spain and serves as the political, economic, and cultural center of the country. Madrid is located in the center of the Iberian Peninsula and is known for its rich history, art, and architecture, including the Royal Palace, the Prado Museum, and the Plaza Mayor.'}}]}]


In [8]:
# "美化 "输出
print(f"Question: {messages[0]['content']}\n")
print(f"Response: {response[0]['choices'][0]['message']['content']}\n")
print(f"Usage: {response[0]['usage']}")

Question: What is the capital of Spain?

Response: The capital of Spain is Madrid. It is the largest city in Spain and serves as the political, economic, and cultural center of the country. Madrid is located in the center of the Iberian Peninsula and is known for its rich history, art, and architecture, including the Royal Palace, the Prado Museum, and the Plaza Mayor.

Usage: {'prompt_tokens': 11, 'completion_tokens': 73, 'total_tokens': 84}
